## **Swiggy Data Science Internship Task**



# Context:

---

In last few years the food e-catering business have grown 
exponentially. This leads to increase in demand for better solutions which involve assigning orders to Delivery Executives optimally. This notebook deals a solution in the similar direction with the given data.


# Porblem: 

---

This problem is an extenstion of Vehicle Routing Problem. The added constraints are the capacity of vehicle, multiple pickup-delivery locations. 

# Solution: 

---

During the last 50+ years a lot of research has been done on solving Vehicle Routing Problem. This notebook deals with one of the solutions which is implemented in the Google *ortools* library.
This problem basically deals with trying out multiple combinations to get the smallest path. 

The alogorithm used in this notebook is local search options which finds an optimal solution. The time complexity is very hight as it usually work well for small number of delieveres(upto 15).

Since the data size is very small (20 deliveries) the implemented algorithm is sufficient to find out the optimal results with given constraints on maximum capacity and multiple pickup-deliveries.


# Tools Used: 

---
Google Developer OR Tools, pandas, math

# Assumptions: 

---

1. Since the distance coordinates are given in (lat,lon) format, I have used the "Haversine" formulat to calculate the distance between any two points.

2. To simplify the data the *order_id* are assigned indices in range [1,20] and correspondingly the *rest_id* are assigned indices in range [21, 40]. Any *rest_id* correspond to *order_id + 20*

3. Each restaurant already have at least 1 delievery executive. Since this algorithm uses a depot station which initially have all the delivery partners. The DE starts from depot -> take order -> deliever -> go back to depot. Since we are dealing with multiple depots and to simplify the problem the ditance from depot to any restaurant or order location is assigned 0 so that it is not taken into consideration. 

4. The solution tries to minimize the maximum distance travelled by any DE and then calculates the total distance travelled by every DE. The ditance parameter is set to the maximum distance between pickup-delivery location which finally also reduces the total distance travelled by all delievery executives.

Relevant comments have been added with the code snippets.

In [14]:
"""Importing Google ortools and other relevant libraries"""
!pip install ortools
import math
from math import cos, asin, sqrt, pi
import pandas as pd
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
"""Haversine formula to calculate distance between two points with lat lon given"""
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

In [16]:
"""Creating data into relevant format to that of the routing model"""
file_path = "/content/sample_data_set_assignment_v6.xlsx"
data = pd.read_excel(file_path);

# Assigning new indices to rest_id and order_id

for i in range(data.shape[0]):
  data['order_id'][i] = i + 1;
  data['rest_id'][i ] = i + 1 + data.shape[0];

# Putting all the location of restaurants and orders in a dictonary to easily calculate distance between any two locations
dict = {}
for i in range(data.shape[0]):
  dict[data['rest_id'][i]] = data['rest_location'][i]
  dict[data['order_id'][i]] = data['cust_location'][i]

cols = 41
rows = cols
mx = 0

# Creating the distance matrix for distance between all pairs of locations
arr = [[0 for i in range(cols)] for j in range(rows)]
for i in range (1, 41):
  for j in range(1, 41):
    c1 = dict[i].split(',')
    c2 = dict[j].split(',')
    arr[i][j] = math.floor(distance(float(c1[0]), float(c1[1]), float(c2[0]), float(c2[1])) * 1000)
    arr[j][i] = arr[i][j]

# Calculating the maximum distance between all pickup-delivery location
for i in range (1, 21):
  c1 = dict[i].split(',')
  c2 = dict[i+20].split(',')
  mx = max(math.floor(distance(float(c1[0]), float(c1[1]), float(c2[0]), float(c2[1])) * 1000), mx)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
# Creating the data model to feed into the pywrapcp RoutingModel routing manager
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = arr
    data['demands'] = [0]*41
    for i in range(1, 21):
      data['demands'][i] = -1;
      data['demands'][i+20] = 1;
    arr2 = [[0 for i in range(2)] for j in range(20)]
    for i in range (0, 20):
      arr2[i][0] = i + 1 + 20;
      arr2[i][1] = i + 1;
    data['pickups_deliveries'] = arr2
    data['vehicle_capacities'] = [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
    data['num_vehicles'] = 20
    data['depot'] = 0
    return data
data = create_data_model()

In [18]:
# Function to print the opitmal paths of every DE and distance travelled with all constraints
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            if(manager.IndexToNode(index) != 0):
              plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            if(manager.IndexToNode(index) != 0):
              route_distance += routing.GetArcCostForVehicle(
                  previous_index, index, vehicle_id)
        # plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))

In [29]:
# Function to initialize the Routing model and calculat the parameters for routing model which minimizes the total cost.
def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        mx,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    # Setting local search metaheuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 1
    search_parameters.log_search = True
    # search_parameters.time_limit.seconds = 300

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(data, manager, routing, assignment)


In [30]:
# Printing the final paths of all DE's
if __name__ == '__main__':
    main()

Route for vehicle 0:
 38 ->  25 ->  18 ->  5 -> Distance of the route: 3172m

Route for vehicle 1:
 35 ->  15 -> Distance of the route: 592m

Route for vehicle 2:
 33 ->  13 -> Distance of the route: 626m

Route for vehicle 3:
 26 ->  6 -> Distance of the route: 1035m

Route for vehicle 4:
 27 ->  37 ->  7 ->  17 -> Distance of the route: 3060m

Route for vehicle 5:
 30 ->  10 -> Distance of the route: 1229m

Route for vehicle 6:
 23 ->  3 -> Distance of the route: 1413m

Route for vehicle 7:
 39 ->  19 -> Distance of the route: 1597m

Route for vehicle 8:
 24 ->  4 -> Distance of the route: 1739m

Route for vehicle 9:
 29 ->  9 -> Distance of the route: 1951m

Route for vehicle 10:
 32 ->  12 -> Distance of the route: 2037m

Route for vehicle 11:
 40 ->  20 -> Distance of the route: 2147m

Route for vehicle 12:
 34 ->  14 -> Distance of the route: 2168m

Route for vehicle 13:
 21 ->  1 -> Distance of the route: 2181m

Route for vehicle 14:
 36 ->  16 -> Distance of the route: 2356m

R

## Results:

The minimum distance to be travelled with 20 DE's availabe is 37.797 km. The total cost of travel will be Rs. 113.
The paths of each DE are mentioned above.

Some possible changes to manipulate the results can be:
1. To minimize the total DE's the maximum distance allowed parameter should be increased accordingly. 

2. The capacity of DE's can be changed in individual basis.

3. Time Constraints can also be added as another dimension in the routing model. 

4. The results obtained depends on the time limit the algorithm is allowed to run. The cost usually reduces with more time allowed. The solution presented here works within 1s. The results obtaied when time limit was set upto 60s were also the same.

Author's Comments:

I have treated this Routing model as a Black Box. While there is lot to explore and read about the solutions for such problem, due to time constraints I decided to proceed this way. I am ready to dive deep into the theory behind the algorithm with sufficient time available.

## THANKS!